# PySpark SQL

### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## 1. SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession
# Create a Session
spark = SparkSession.builder.getOrCreate()
# How is my session?
print(spark)

In [2]:
spark

#### Passing other options to spark session:
    
    

In [59]:
spark = SparkSession.builder.config('someoption.key','somevalue').getOrCreate()

We can check option values in the resulting session like this:

In [60]:
spark.sparkContext.getConf().getAll()

[('someoption.key', 'somevalue'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '38059'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.app.id', 'local-1558192457954'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.ui.showConsoleProgress', 'true')]

## 2. Creating DataFrames

SparkSession.createDataFrame: from a RDD, a list or a pandas.DataFrame.

### 2.1 Zip 

In [61]:
# Create the rows for the Dataframe
import random
random.seed(42)
ids= range(5)
positions=[random.choice(['mechanic','sales','manager']) for id_ in ids]
print(positions)
# Join ids with positions through zip command
rows = zip(ids,positions)
print(rows)

['manager', 'mechanic', 'mechanic', 'manager', 'sales']


In [62]:
# Create the Dataframe
df = spark.createDataFrame(rows)

In [63]:
# Show the content. It's similar to .head() in Pandas 
df.show()

+---+--------+
| _1|      _2|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+



In [64]:
# Collect is another way to see the Dataframe
df.collect()

[Row(_1=0, _2='manager'),
 Row(_1=1, _2='mechanic'),
 Row(_1=2, _2='mechanic'),
 Row(_1=3, _2='manager'),
 Row(_1=4, _2='sales')]

### 2.2 Row

In [65]:
from pyspark.sql import Row

rows = [Row(id=id_, position=position_) for id_, position_ in zip(ids,positions)]
type(rows)

list

In [66]:
df = spark.createDataFrame(rows)
df.show()

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+



### 2.3 Read

```python
# Read a file from a CSV
df = spark.read.csv(file_path,header=True, inferSchema=True)
# Read a file from a CSV
df = spark.read.format('csv').options(header=True, inferSchema=True).load(file_path)
```

In [67]:
# Infer Schema
df = spark.read.csv(path,header=False,inferSchema=True)
df.printSchema()

root
 |-- _c0: long (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)



In [68]:
# All our columns are strings by default
path = '/home/eduardo/Repos/Master_DataScience/19_Spark/coupon150720.csv'
df = spark.read.csv(path,header=False).limit(10)
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)



## 3. Inferring and Specifying Schemas

In [69]:
df = df.select('_c0','_c1','_c2','_c6')
df.show(5)

+--------------+---+---+------+
|           _c0|_c1|_c2|   _c6|
+--------------+---+---+------+
|79062005698500|  1|MAA| 56.79|
|79062005698500|  2|AUH| 84.34|
|79062005924069|  1|CJB|  60.0|
|79065668570385|  1|DEL|160.63|
|79065668737021|  1|AUH|152.46|
+--------------+---+---+------+
only showing top 5 rows



In [70]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c6: string (nullable = true)



In [71]:
from pyspark.sql import types

# Column Types
fields = [types.StructField('_c0',types.StringType()),
          types.StructField('_c1',types.IntegerType()),
          types.StructField('_c2',types.StringType()),
          types.StructField('_c6',types.FloatType())]

# Create my Schema
my_schema = types.StructType(fields)

# Read Dataframe
df = spark.read.csv(path, schema=my_schema).select('_c0','_c1','_c2','_c6')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c6: float (nullable = true)



## 4. Filtering and Selecting

### 4.1 Selecting

In [72]:
df.select('_c0').show(5)

+--------------+
|           _c0|
+--------------+
|79062005698500|
|79062005698500|
|79062005924069|
|79065668570385|
|79065668737021|
+--------------+
only showing top 5 rows



In [73]:
df.select(df['_c0']).show(5)

+--------------+
|           _c0|
+--------------+
|79062005698500|
|79062005698500|
|79062005924069|
|79065668570385|
|79065668737021|
+--------------+
only showing top 5 rows



In [74]:
from pyspark.sql import functions as f

df.select(f.col('_c0')).show(5)

+--------------+
|           _c0|
+--------------+
|79062005698500|
|79062005698500|
|79062005924069|
|79065668570385|
|79065668737021|
+--------------+
only showing top 5 rows



In [75]:
df.select(df._c0).show(5)

+--------------+
|           _c0|
+--------------+
|79062005698500|
|79062005698500|
|79062005924069|
|79065668570385|
|79065668737021|
+--------------+
only showing top 5 rows



### 4.2 Filtering

In [77]:
# Read CSV
df = spark.read.csv(path,header=False,inferSchema=True).limit(10)
# Filter
filtered = df.filter(df['_c2'] == 'AUH')
filtered.show()

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+



In [79]:
# Filter
filtered2 = df.filter(df['_c6'] > 50)
filtered2.show()

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
|79062006192650|  1|RPR|BOM| 9W| 9W|  68.5|USD|  1|  K|   K|2202|150720|  OK|IAE0|
|79062006192650|  2|BOM|RPR| 9W| 9W|  68.5|USD|  1|  H|   H|0377|150721|  OK|IAE0|
|79062005733853|  1|DEL|DED| 9W| 9W| 56.16|USD|  1|  V|   V|2839|150801|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+



In [83]:
# Where is similar to Filter
whered = df.where(f.col('_c8') >= 2)
whered.show()

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+



## 5. Adding Columns

In [88]:
df.withColumn('price',f.col('_c6')*100).show()

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+-------+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|  price|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+-------+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0| 5679.0|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0| 8434.0|
|79062005924069|  1|CJB|MAA|  9W|  9W|  60.0|USD|  1|   H|   H|2768|150721|  OK|IAA0| 6000.0|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0|16063.0|
|79065668737021|  1|AUH|IXE|  9W|  9W|152.46|USD|  1|   V|   V|0501|150803|  OK|INA0|15246.0|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IAE0| 6850.0|
|79062006192650|  2|BOM|RPR|  9W|  9W|  68.5|USD|  1|   H|   H|0377|150721|  OK|IAE0| 6850.0|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|  

In [94]:
df = df.withColumn('_c1', f.col('_c1').cast(types.IntegerType()))
df.show()

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA|  9W|  9W|  60.0|USD|  1|   H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE|  9W|  9W|152.46|USD|  1|   V|   V|0501|150803|  OK|INA0|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IAE0|
|79062006192650|  2|BOM|RPR|  9W|  9W|  68.5|USD|  1|   H|   H|0377|150721|  OK|IAE0|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|   V|2839|150801|  OK|INA0|
|79062005836987|  1|ATL|LGA|  AA|  AA|  28.3|USD|  1| 

## 6. User Defined Functions UDF

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (udfs), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [95]:
import math
math.log1p(0)

0.0

In [97]:
# df.select(math.log1p('_c1')) this is going to return an error
# we need to define an UDF to do this

In [98]:
# Create Function and Define UDF
def log1p_fun(num):
    return math.log1p(num)

udf_log1p = f.udf(log1p_fun)

In [122]:
df.withColumn('newColumnUDF',udf_log1p(f.col('_c1'))).select('_c0','_c2','_c3','_c6','newColumnUDF').show()    # f.col('_c1') == df['_c1']

+--------------+---+---+------+------------------+
|           _c0|_c2|_c3|   _c6|      newColumnUDF|
+--------------+---+---+------+------------------+
|79062005698500|MAA|AUH| 56.79|0.6931471805599453|
|79062005698500|AUH|CDG| 84.34|1.0986122886681096|
|79062005924069|CJB|MAA|  60.0|0.6931471805599453|
|79065668570385|DEL|DXB|160.63|0.6931471805599453|
|79065668737021|AUH|IXE|152.46|0.6931471805599453|
|79062006192650|RPR|BOM|  68.5|0.6931471805599453|
|79062006192650|BOM|RPR|  68.5|1.0986122886681096|
|79062005733853|DEL|DED| 56.16|0.6931471805599453|
|79062005836987|ATL|LGA|  28.3|0.6931471805599453|
|79062005836987|LGA|EWR|   0.0|1.0986122886681096|
+--------------+---+---+------+------------------+



In [123]:
# UDF to get odd letters
def odd_letters(word):
    return word[::2]
udf_odd = f.udf(odd_letters, returnType=types.StringType()) # It will be StringType by default

df.withColumn('newColOddLetters',udf_odd(df['_c2'])).select('_c0','_c2','_c3','_c6','newColOddLetters').show()

+--------------+---+---+------+----------------+
|           _c0|_c2|_c3|   _c6|newColOddLetters|
+--------------+---+---+------+----------------+
|79062005698500|MAA|AUH| 56.79|              MA|
|79062005698500|AUH|CDG| 84.34|              AH|
|79062005924069|CJB|MAA|  60.0|              CB|
|79065668570385|DEL|DXB|160.63|              DL|
|79065668737021|AUH|IXE|152.46|              AH|
|79062006192650|RPR|BOM|  68.5|              RR|
|79062006192650|BOM|RPR|  68.5|              BM|
|79062005733853|DEL|DED| 56.16|              DL|
|79062005836987|ATL|LGA|  28.3|              AL|
|79062005836987|LGA|EWR|   0.0|              LA|
+--------------+---+---+------+----------------+



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [124]:
udf_log1p_typed = f.udf(math.log1p, types.FloatType())
df.withColumn('newColUdf', udf_log1p_typed(df['_c1'])).select('_c0','_c2','_c3','_c6','newColUdf').show()

+--------------+---+---+------+---------+
|           _c0|_c2|_c3|   _c6|newColUdf|
+--------------+---+---+------+---------+
|79062005698500|MAA|AUH| 56.79|0.6931472|
|79062005698500|AUH|CDG| 84.34|1.0986123|
|79062005924069|CJB|MAA|  60.0|0.6931472|
|79065668570385|DEL|DXB|160.63|0.6931472|
|79065668737021|AUH|IXE|152.46|0.6931472|
|79062006192650|RPR|BOM|  68.5|0.6931472|
|79062006192650|BOM|RPR|  68.5|1.0986123|
|79062005733853|DEL|DED| 56.16|0.6931472|
|79062005836987|ATL|LGA|  28.3|0.6931472|
|79062005836987|LGA|EWR|   0.0|1.0986123|
+--------------+---+---+------+---------+



In [107]:
f.udf(lambda x: -x)

<function __main__.<lambda>(x)>

In [125]:
def prices(num):
    if num > 100:
        return 'Expensive'
    else:
        return 'Cheap'
    
udf_prices = f.udf(prices, types.StringType())

df.withColumn('prices',udf_prices(f.col('_c6'))).select('_c0','_c2','_c3','_c6','prices').show()

+--------------+---+---+------+---------+
|           _c0|_c2|_c3|   _c6|   prices|
+--------------+---+---+------+---------+
|79062005698500|MAA|AUH| 56.79|    Cheap|
|79062005698500|AUH|CDG| 84.34|    Cheap|
|79062005924069|CJB|MAA|  60.0|    Cheap|
|79065668570385|DEL|DXB|160.63|Expensive|
|79065668737021|AUH|IXE|152.46|Expensive|
|79062006192650|RPR|BOM|  68.5|    Cheap|
|79062006192650|BOM|RPR|  68.5|    Cheap|
|79062005733853|DEL|DED| 56.16|    Cheap|
|79062005836987|ATL|LGA|  28.3|    Cheap|
|79062005836987|LGA|EWR|   0.0|    Cheap|
+--------------+---+---+------+---------+



## 7. Grouping

In [112]:
df.groupBy('_c2').mean('_c6').show()

+---+--------+
|_c2|avg(_c6)|
+---+--------+
|MAA|   56.79|
|AUH|   118.4|
|CJB|    60.0|
|DEL| 108.395|
|RPR|    68.5|
|BOM|    68.5|
|ATL|    28.3|
|LGA|     0.0|
+---+--------+



In [113]:
stats = df.groupBy('_c2').agg(f.mean('_c6').alias('average'),
                              f.stddev_pop('_c6').alias('std'),
                              f.count('_c6').alias('count')
                              ).show()

+---+-------+------+-----+
|_c2|average|   std|count|
+---+-------+------+-----+
|MAA|  56.79|   0.0|    1|
|AUH|  118.4| 34.06|    2|
|CJB|   60.0|   0.0|    1|
|DEL|108.395|52.235|    2|
|RPR|   68.5|   0.0|    1|
|BOM|   68.5|   0.0|    1|
|ATL|   28.3|   0.0|    1|
|LGA|    0.0|   0.0|    1|
+---+-------+------+-----+



In [115]:
df.groupBy(['_c2','_c8']).agg(f.mean('_c6').alias('average')).show()

+---+---+-------+
|_c2|_c8|average|
+---+---+-------+
|MAA|  1|  56.79|
|AUH|  1|  118.4|
|CJB|  1|   60.0|
|DEL|  2| 160.63|
|RPR|  1|   68.5|
|BOM|  1|   68.5|
|DEL|  1|  56.16|
|ATL|  1|   28.3|
|LGA|  1|    0.0|
+---+---+-------+



## 8. Joins

In [117]:
valuesA = [('MAA','Chennai'),('AUH','Abu Dabi'),('CJB','Coimbatore'),('DEL','Delhi'),('RPR','Raipur'),('BOM','Bombai'),('ATL','Atlanta'),('LGA','La Guardia')]
TableA = spark.createDataFrame(valuesA,['code','name'])
TableA.show()

+----+----------+
|code|      name|
+----+----------+
| MAA|   Chennai|
| AUH|  Abu Dabi|
| CJB|Coimbatore|
| DEL|     Delhi|
| RPR|    Raipur|
| BOM|    Bombai|
| ATL|   Atlanta|
| LGA|La Guardia|
+----+----------+



In [118]:
df.join(TableA, on=df['_c2']==TableA['code']).show()

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+----+----------+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|code|      name|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+----+----------+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0| MAA|   Chennai|
|79065668737021|  1|AUH|IXE|  9W|  9W|152.46|USD|  1|   V|   V|0501|150803|  OK|INA0| AUH|  Abu Dabi|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0| AUH|  Abu Dabi|
|79062005924069|  1|CJB|MAA|  9W|  9W|  60.0|USD|  1|   H|   H|2768|150721|  OK|IAA0| CJB|Coimbatore|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|   V|2839|150801|  OK|INA0| DEL|     Delhi|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0| DEL|     Delhi|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IA

In [120]:
df.join(TableA, on=df['_c3']==TableA['code']).show()

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+----+----------+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|code|      name|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+----+----------+
|79062005924069|  1|CJB|MAA| 9W| 9W| 60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0| MAA|   Chennai|
|79062005698500|  1|MAA|AUH| 9W| 9W|56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0| AUH|  Abu Dabi|
|79062006192650|  2|BOM|RPR| 9W| 9W| 68.5|USD|  1|  H|   H|0377|150721|  OK|IAE0| RPR|    Raipur|
|79062006192650|  1|RPR|BOM| 9W| 9W| 68.5|USD|  1|  K|   K|2202|150720|  OK|IAE0| BOM|    Bombai|
|79062005836987|  1|ATL|LGA| AA| AA| 28.3|USD|  1|  V|   V|3237|150903|  OK|INB0| LGA|La Guardia|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+----+----------+



In [121]:
df.join(TableA, on=df['_c3']==TableA['code'],how='left').show()

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+----+----------+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|code|      name|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+----+----------+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0| AUH|  Abu Dabi|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0|null|      null|
|79062005836987|  1|ATL|LGA|  AA|  AA|  28.3|USD|  1|   V|   V|3237|150903|  OK|INB0| LGA|La Guardia|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IAE0| BOM|    Bombai|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0|null|      null|
|79062005836987|  2|LGA|EWR|null|null|   0.0|USD|  1|null|null|VOID|  null|null|INA0|null|      null|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|   V|2839|150801|  OK|IN